In [1]:
import numpy as np
import utils
import pandas as pd

In [2]:
synth_simple = utils.load_synthetic(ver="simple")
adult = utils.load_adult(scaler = True)
RESPONSE = "income"

In [5]:
adult["train"].columns

Index(['age', 'education', 'education-num', 'occupation', 'relationship',
       'capital-gain', 'capital-loss', 'hours-per-week', 'native-country',
       'race', 'gender', 'marital-status', 'workclass', 'income'],
      dtype='object')

In [6]:
#percentiles = [p for p in range(0,20,2)]+[p for p in range(20,80, 10)]
percentiles = [p for p in range(60,80, 10)]

all_results = {"Full data": {}, "Averaged results": {} }
missing = ["workclass", "gender", "relationship"]
sensitive = ["gender", "race"]
RUNS = 2
for miss in missing:
    for sens in sensitive:
        print(miss+sens)
        adult_results = utils.test_bench(data = "adult", pred = RESPONSE, missing = miss, sensitive=sens,
                        percentiles = percentiles, n_runs=RUNS, differencing=True)
        

        all_results["Full data"][miss+"_"+sens+"_"+"adult"] = adult_results["Full data"]
        all_results["Averaged results"][miss+"_"+sens+"_"+"adult"] = adult_results["Averaged results"]
        

        

workclassgender


100%|██████████| 2/2 [02:46<00:00, 83.03s/it]


workclassrace


100%|██████████| 2/2 [03:21<00:00, 100.77s/it]


gendergender


...

relationshipgender


100%|██████████| 2/2 [01:49<00:00, 54.66s/it]


relationshiprace


100%|██████████| 2/2 [02:02<00:00, 61.50s/it]


<Figure size 1440x792 with 0 Axes>

In [8]:
#Synth
miss = "x_2"
sensitive = "x_1"
RESPONSE = "y"
synth_results = utils.test_bench(data = "simple", pred = RESPONSE, missing = miss, sensitive=sensitive,
                            percentiles = percentiles, n_runs=RUNS, differencing=True)
all_results["Full data"][miss+"_"+sensitive+"_"+"synth"] = synth_results["Full data"]
all_results["Averaged results"][miss+"_"+sensitive+"_"+"synth"] = synth_results["Averaged results"]

100%|██████████| 2/2 [00:17<00:00,  8.64s/it]


<Figure size 1440x792 with 0 Axes>

In [ ]:
def sigmoid(x, alpha):
    z = np.exp(-x+alpha)
    sig = 1 / (1 + z)
    return sig

size = 10000
x_1 = np.random.binomial(1, 0.45, size=size)
x_2 = np.random.binomial(1, 0.65, size=size)
x_3 = np.random.normal(0,1,size)
y = np.around(sigmoid(x_1*0.3+x_2+x_3, alpha=0.8)).astype(int)
synth_cat = pd.DataFrame({"y": y, "x_1": x_1, "x_2": x_2, "x_3": x_3})
synth_cat_test = synth_cat.iloc[:round(0.333*size), :]
synth_cat_train = synth_cat.iloc[round(0.333*size):, :]
synth_cat["y"].sum()

24763

In [9]:
import json
from pathlib import Path
with open(Path("raw_data/test.json"), 'w') as f:
    json.dump(all_results, f)